In [26]:
# To adjust camer angles

import carla
import random
import cv2
import numpy as np

client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
world = client.get_world() 

# blueprint will access to all blueprints to create objects (vehicles, people, etc.)
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

camera_bp = bp_lib.find('sensor.camera.rgb')

IM_WIDTH = 256*4
IM_HEIGHT = 256*3
        
camera_bp.set_attribute('image_size_x', f'{IM_WIDTH}')
camera_bp.set_attribute('image_size_y', f'{IM_HEIGHT}')
camera_bp.set_attribute('fov', '110')

camera_init_trans = carla.Transform(carla.Location(x = 1.5, z = 1.6 ))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

def camera_callback(image, data_dict):
    image_data = np.array(image.raw_data)
    image_rgb = image_data.reshape((image.height, image.width, 4))[:, :, :3]
    data_dict['image'] = image_rgb

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h, image_w, 4))}
camera.listen(lambda image: camera_callback(image, camera_data))

while True:
    frame = camera_data['image']
    cv2.imshow('Carla', frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        cv2.destroyAllWindows()  # Close all OpenCV windows
        break